# Day 8
Find the description of the problem [here](https://adventofcode.com/2025/day/8)!

## Part 1

Puzzle input:

In [335]:
with open("input_files/day_08.txt") as input_file:
    input = input_file.read()

Test input:

In [336]:
num_connections = 1000

# # Comment this cell to use the puzzle input instead of the test input
# input = """162,817,812
# 57,618,57
# 906,360,560
# 592,479,940
# 352,342,300
# 466,668,158
# 542,29,236
# 431,825,988
# 739,650,466
# 52,470,668
# 216,146,977
# 819,987,18
# 117,168,530
# 805,96,715
# 346,949,466
# 970,615,88
# 941,993,340
# 862,61,35
# 984,92,344
# 425,690,689"""
# num_connections = 10

Parse the input:

In [337]:
boxes = [tuple(map(int, line.split(","))) for line in input.splitlines()]

In [338]:
from math import sqrt

def dist(box1, box2):
    x1, y1, z1 = box1
    x2, y2, z2 = box2
    return sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2)

After a lot of effort implementing a brute force solution that worked for the example input, I realized it was extremely slow on the actual input, as it had to perform 1 billion iterations. Fortunately I thought about itertools product, which made the code much much faster.

In [339]:
import itertools

all_paired_boxes = list(itertools.product(boxes, boxes))
all_paired_boxes = sorted(all_paired_boxes, key=lambda x: dist(*x))
paired_boxes = []
for pair in all_paired_boxes:
    box1, box2 = pair
    if box1 == box2:
        continue
    if (box1, box2) in paired_boxes or (box2, box1) in paired_boxes:
        continue
    paired_boxes.append(pair)
    if len(paired_boxes) == num_connections:
        break

Once we have the first `num_connections` connections, it's a matter of creating circuits. Here I struggled because at first I wasn't considering what would happen if a pair made two circuits to connect to each other, but after furiously debugging for ~~a few~~ many minutes, I finally arrived at this solution:

In [340]:
circuits = []
connections = 0
for box1, box2 in paired_boxes:
    if connections == num_connections:
        break
    to_add = []
    add_to = []
    for circuit in circuits:
        if box1 in circuit and box2 in circuit:
            break
        elif box1 in circuit:
            to_add.append(box2)
            add_to.append(circuit)
        elif box2 in circuit:
            to_add.append(box1)
            add_to.append(circuit)
    else:
        if len(to_add) == 0:
            circuits.append([box1, box2])
        else:
            if len(add_to) == 2:
                circuits.remove(add_to[0])
                circuits.remove(add_to[1])
                circuits.append(list(set(add_to[0] + add_to[1])))
            else:
                add_to[0].append(to_add[0])
    connections += 1

circuits = sorted(circuits, key=lambda x: -len(x))

result = 1
for i in range(3):
    result *= len(circuits[i])
print(f"The result of multiplying the size of the three largest circuits is {result}.")

The result of multiplying the size of the three largest circuits is 133574.


## Part 2

Starting from a copy-paste of the part 1 code, I removed the `num_connections` connection limit, and just cut out the iterations once there was a single circuit:

In [341]:
paired_boxes = []
circuits = []
for pair in all_paired_boxes:
    box1, box2 = pair
    if box1 == (216,146,977) and box2 == (117,168,530) or box1 == (117,168,530) and box2 == (216,146,977):
        pass
    if box1 == box2:
        continue
    if (box1, box2) in paired_boxes or (box2, box1) in paired_boxes:
        continue
    paired_boxes.append(pair)

    to_add = []
    add_to = []
    for circuit in circuits:
        if box1 in circuit and box2 in circuit:
            break
        elif box1 in circuit:
            to_add.append(box2)
            add_to.append(circuit)
        elif box2 in circuit:
            to_add.append(box1)
            add_to.append(circuit)
    else:
        if len(to_add) == 0:
            circuits.append([box1, box2])
        else:
            if len(add_to) == 2:
                circuits.remove(add_to[0])
                circuits.remove(add_to[1])
                circuits.append(list(set(add_to[0] + add_to[1])))
            else:
                add_to[0].append(to_add[0])
    if len(circuits[0]) == len(boxes):
        last_boxes = pair
        break
    
result = last_boxes[0][0] * last_boxes[1][0]
print(f"The multiplication of the X coordinates of the last two boxes is {result}.")


The multiplication of the X coordinates of the last two boxes is 2435100380.
